In [1]:
# transcript_location_to_genome_location()
gff="/home/wuyou/Projects/Osativa_nanopore/ref/Osativa_323_v7.0.gene_exons.gtf"

fasta="/home/wuyou/Projects/Osativa_nanopore/ref/Osativa_323_v7.0.fa"

sequence_dict={}


chr="Chr0"
sequence=""
with open(fasta) as f:
    for line in f:

        line=line.rstrip()
        if ">" in line:
            sequence_dict[chr]=sequence
            sequence=""
            
            chr=line[1:]
            print(chr) 
        else:
            sequence+=line
sequence_dict[chr]=sequence    



exon_dict={}
chr_dict={}
strand_dict={}
with open(gff) as f:
    for line in f:
        type=line.split("\t")[2]
        if type=="exon":
            chr=line.split('\t')[0]
            transcript_id=line.split('"')[1]
            start=int(line.split("\t")[3])
            end=int(line.split("\t")[4])
            strand=line.split("\t")[6]
            if transcript_id not in exon_dict:
                exon_dict[transcript_id]=[]
                chr_dict[transcript_id]=chr
                strand_dict[transcript_id]=strand
            exon_dict[transcript_id].append([start,end])


def revere_complement(seq):
    seq=seq.replace("A","t")
    seq=seq.replace("C","g")
    seq=seq.replace("G","c")
    seq=seq.replace("T","a")
    return seq.upper()[::-1]


import time
transcript_location="../demo_data/test3.prediction.tsv"
genome_location="../demo_data/test3.prediction.genome_loc.tsv"
out=open(genome_location,"w")

count=0
with open(transcript_location) as f:
    for line in f:
        count+=1
        if count % 1e7==0:
            print(count)
        items=line.rstrip().split("\t")
        transcript_id=items[0]
        transcript_location=int(items[1])
        motif=items[2]
        mod=items[4]
        probability=items[5]
        
        exons=exon_dict[transcript_id]
        strand=strand_dict[transcript_id]
        if strand=="+":
            summation=0
            for exon in exons:
                start=exon[0]
                end=exon[1]
                if transcript_location>summation+end-start+1:
                    summation=summation+end-start+1
                else:
                    chr_location=transcript_location-summation+start-1
                    break

            print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" %(transcript_id,transcript_location,motif,chr_dict[transcript_id],chr_location,sequence_dict[chr_dict[transcript_id]][chr_location-3:chr_location+2],mod,probability),file=out)
        if strand=="-":
            summation=0
            for exon in exons[::-1]:       #reverse
                start=exon[0]
                end=exon[1]
                if transcript_location>summation+end-start+1:
                    summation=summation+end-start+1
                else:
                    chr_location=end-(transcript_location-summation)+1
                    break

            print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" %(transcript_id,transcript_location,motif,chr_dict[transcript_id],chr_location,revere_complement(sequence_dict[chr_dict[transcript_id]][chr_location-3:chr_location+2]),mod,probability),file=out)



Chr1
Chr2
Chr3
Chr4
Chr5
Chr6
Chr7
Chr8
Chr9
Chr10
Chr11
Chr12
ChrUn
ChrSy
